# Predict Price of Flight Tickets

## Attributes:

Data contains the following features:  
   * Airline - str, name of operating airline  
   * Date-of-Journey - datetime, date of departure  
   * Source - str, name of city flight departs from  
   * Destination - str, destination city  
   * Route - str, string of cities that flights passes through  
   * Dep_Time - timestamp, time of departure  
   * Arrival_Tiem - timestamp, time of arrival  
   * Duration - timedifference, length of flight in hh:mm  
   * Stops - str, number of stops in flight journey (0, 2]  
   * Additional_Info - str, categorical data including info on in flight amenities, etc  
   * Price - int, dependendent variable, cost of flight  
    
Training set contains 10683 rows, test set contains 2671 rows.  
 
 ## Preprocessing Plan
 
 I will use a tiered approach to preprocessing the data set. I will do intial preprocessing in Excel. These will be easy task like converting the 'Date_of_Jorney' column from a string in form dd/mm/YYYY into canonical format (YYYY-mm-dd). Categorical data will be handled with pandas and sklearn.preprocessing. Scaling will be conducted in python using sklearn's numeric scalers.

## Import Necessary Modules

In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

## Import Data

In [2]:
# set path, read file to constuct pandas dataframe
path = r'20190310_Data_Train.xlsx'
df = pd.read_excel(path, header=0, dtype='str')
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,2019-03-24 00:00:00,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,2019-05-01 00:00:00,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,2019-06-09 00:00:00,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,2019-05-12 00:00:00,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,2019-03-01 00:00:00,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


## Quick Overview of Data

In [3]:
# check that all the data was properly imported
print('shape:', df.shape, '\n')
print('dtpyes: \n' + str(df.dtypes))

shape: (10683, 11) 

dtpyes: 
Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price              object
dtype: object


In [4]:
# Get counts of categorical features
print('number of airlines:', len(df['Airline'].unique()))
print('number of sources:', len(df['Source'].unique()))
print('number of destinations:', len(df['Destination'].unique()))
print('number of stops:', len(df['Total_Stops'].unique()))
print('number of Additional Info:', len(df['Additional_Info'].unique()))

number of airlines: 12
number of sources: 5
number of destinations: 6
number of stops: 6
number of Additional Info: 10


In [5]:
# Check range of dates in df
print('Date range:', pd.to_datetime(df['Date_of_Journey'].max()) - pd.to_datetime(df['Date_of_Journey'].min()))
print('Latest Date:', df['Date_of_Journey'].max())
print('Earliest Date:', df['Date_of_Journey'].min())
print(df['Date_of_Journey'].describe())

Date range: 118 days 00:00:00
Latest Date: 2019-06-27 00:00:00
Earliest Date: 2019-03-01 00:00:00
count                   10683
unique                     40
top       2019-05-18 00:00:00
freq                      504
Name: Date_of_Journey, dtype: object


40 unique flight dates covering a span of 118 days. Maybe split into weekly, or bi-weekly intervals for model

In [6]:
# Price analysis
df = df.astype(dtype={'Price': 'int64'})
print(df['Price'].describe())

count    10683.000000
mean      9087.064121
std       4611.359167
min       1759.000000
25%       5277.000000
50%       8372.000000
75%      12373.000000
max      79512.000000
Name: Price, dtype: float64


In [7]:
df.sort_values(by='Price', ascending=False).head(25)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
2924,Jet Airways Business,2019-03-01 00:00:00,Banglore,New Delhi,BLR → BOM → DEL,05:45,11:25,5h 40m,1 stop,Business class,79512
5372,Jet Airways Business,2019-03-01 00:00:00,Banglore,New Delhi,BLR → BOM → DEL,05:45,12:25,6h 40m,1 stop,Business class,62427
10364,Jet Airways Business,2019-03-01 00:00:00,Banglore,New Delhi,BLR → MAA → DEL,09:45,14:25,4h 40m,1 stop,Business class,57209
1478,Jet Airways,2019-03-18 00:00:00,Banglore,New Delhi,BLR → BOM → DEL,18:40,00:45 16 Mar,6h 5m,1 stop,No info,54826
5439,Jet Airways,2019-03-01 00:00:00,Banglore,New Delhi,BLR → BOM → DEL,16:55,23:00,6h 5m,1 stop,No info,54826
2618,Jet Airways,2019-03-18 00:00:00,Banglore,New Delhi,BLR → BOM → DEL,22:50,05:05 16 Mar,6h 15m,1 stop,No info,54826
9715,Jet Airways Business,2019-03-06 00:00:00,Delhi,Cochin,DEL → ATQ → BOM → COK,20:05,04:25 07 Mar,8h 20m,2 stops,No info,52285
657,Jet Airways Business,2019-03-01 00:00:00,Banglore,New Delhi,BLR → BOM → DEL,05:45,10:45,5h,1 stop,No info,52229
7351,Jet Airways Business,2019-03-03 00:00:00,Delhi,Cochin,DEL → ATQ → BOM → COK,20:05,04:25 04 Mar,8h 20m,2 stops,No info,46490
396,Multiple carriers,2019-03-01 00:00:00,Delhi,Cochin,DEL → BOM → COK,12:50,19:15,6h 25m,1 stop,No info,36983


Jet Airways holds many of the most expensive flights. Check to see how the descriptive statistics of the flights of Jet Airways compare to the other categories

In [8]:
df[(df['Airline'].isin(['Jet Airways Business'])) | (df['Airline'].isin(['Jet Airways']))]['Price'].describe()

count     3855.000000
mean     11716.631128
std       4656.078505
min       1840.000000
25%       9134.000000
50%      11467.000000
75%      14151.000000
max      79512.000000
Name: Price, dtype: float64

# Preprocessing

## Strategy

   * Drop 'Arrival_Time': Arrival Time can be derived by departure time and duration. Keeping it in the data set introduces a redudancy of information.  
   * Explore Variances of different categorica features to determine groupings  
   * Convert Duration to min or hours  
   * Scale numeric features with normalized dist.  
   * I'd like to reduce the number of unique categorical features as much as possible before building a model  
   * Convert Departure time into hourly blocks
   * Search for outliers in 'Prices'  
   
### ANOVA

In [9]:
import scipy.stats as stats
from statsmodels.formula.api import ols

In [10]:
# Airline
anova = ols('Price ~ C(Airline)', data=df).fit()
anova.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.411
Model:                            OLS   Adj. R-squared:                  0.410
Method:                 Least Squares   F-statistic:                     676.6
Date:                Mon, 11 Mar 2019   Prob (F-statistic):               0.00
Time:                        16:42:09   Log-Likelihood:            -1.0246e+05
No. Observations:               10683   AIC:                         2.049e+05
Df Residuals:                   10671   BIC:                         2.050e+05
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
===================================================================================================================
                                                      coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------
Intercept                                        5590.2602    198.270     28.195      0.000    5201.614    5978.906
C(Airline)[T.Air India]                          4020.9504    215.566     18.653      0.000    3598.401    4443.500
C(Airline)[T.GoAir]                               270.7965    322.415      0.840      0.401    -361.196     902.789
C(Airline)[T.IndiGo]                               83.4227    213.118      0.391      0.695    -334.328     501.173
C(Airline)[T.Jet Airways]                        6053.6632    206.323     29.341      0.000    5649.232    6458.094
C(Airline)[T.Jet Airways Business]               5.277e+04   1459.228     36.162      0.000    4.99e+04    5.56e+04
C(Airline)[T.Multiple carriers]                  5312.4179    223.150     23.806      0.000    4875.001    5749.834
C(Airline)[T.Multiple carriers Premium economy]  5828.5860   1001.969      5.817      0.000    3864.540    7792.632
C(Airline)[T.SpiceJet]                          -1251.9753    233.755     -5.356      0.000   -1710.178    -793.772
C(Airline)[T.Trujet]                            -1450.2602   3546.762     -0.409      0.683   -8402.574    5502.053
C(Airline)[T.Vistara]                            2206.0885    255.912      8.620      0.000    1704.453    2707.724
C(Airline)[T.Vistara Premium economy]            3372.0731   2054.113      1.642      0.101    -654.371    7398.517
==============================================================================
Omnibus:                     4174.988   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            51933.380
Skew:                           1.523   Prob(JB):                         0.00
Kurtosis:                      13.363   Cond. No.                         115.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [11]:
# Source
anova = ols('Price ~ C(Source)', data=df).fit()
anova.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.134
Model:                            OLS   Adj. R-squared:                  0.134
Method:                 Least Squares   F-statistic:                     413.1
Date:                Mon, 11 Mar 2019   Prob (F-statistic):               0.00
Time:                        16:42:09   Log-Likelihood:            -1.0451e+05
No. Observations:               10683   AIC:                         2.090e+05
Df Residuals:                   10678   BIC:                         2.091e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept             8017.4643     91.570     87.556      0.000    7837.970    8196.959
C(Source)[T.Chennai] -3227.5719    238.195    -13.550      0.000   -3694.478   -2760.666
C(Source)[T.Delhi]    2521.9748    111.559     22.607      0.000    2303.298    2740.652
C(Source)[T.Kolkata]  1140.9251    121.662      9.378      0.000     902.445    1379.405
C(Source)[T.Mumbai]  -2957.7555    186.589    -15.852      0.000   -3323.505   -2592.006
==============================================================================
Omnibus:                     6582.065   Durbin-Watson:                   1.989
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           188833.276
Skew:                           2.473   Prob(JB):                         0.00
Kurtosis:                      22.994   Cond. No.                         7.15
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
# Destination
anova = ols('Price ~ C(Destination)', data=df).fit()
anova.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.242
Model:                            OLS   Adj. R-squared:                  0.242
Method:                 Least Squares   F-statistic:                     683.3
Date:                Mon, 11 Mar 2019   Prob (F-statistic):               0.00
Time:                        16:42:09   Log-Likelihood:            -1.0380e+05
No. Observations:               10683   AIC:                         2.076e+05
Df Residuals:                   10677   BIC:                         2.077e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                    9158.3894     74.926    122.232      0.000    9011.520    9305.258
C(Destination)[T.Cochin]     1381.0496     95.741     14.425      0.000    1193.379    1568.720
C(Destination)[T.Delhi]     -4014.4708    135.481    -29.631      0.000   -4280.038   -3748.903
C(Destination)[T.Hyderabad] -4098.6807    169.523    -24.178      0.000   -4430.977   -3766.384
C(Destination)[T.Kolkata]   -4368.4970    218.900    -19.957      0.000   -4797.582   -3939.412
C(Destination)[T.New Delhi]  2759.3273    151.352     18.231      0.000    2462.649    3056.005
==============================================================================
Omnibus:                     5977.286   Durbin-Watson:                   1.989
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           171586.561
Skew:                           2.146   Prob(JB):                         0.00
Kurtosis:                      22.159   Cond. No.                         6.88
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
# Total_Stops
anova = ols('Price ~ C(Total_Stops)', data=df).fit()
anova.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.402
Model:                            OLS   Adj. R-squared:                  0.401
Method:                 Least Squares   F-statistic:                     1791.
Date:                Mon, 11 Mar 2019   Prob (F-statistic):               0.00
Time:                        16:42:09   Log-Likelihood:            -1.0253e+05
No. Observations:               10682   AIC:                         2.051e+05
Df Residuals:                   10677   BIC:                         2.051e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                   1.059e+04     47.577    222.672      0.000    1.05e+04    1.07e+04
C(Total_Stops)[T.2 stops]   2121.6843    103.152     20.568      0.000    1919.487    2323.882
C(Total_Stops)[T.3 stops]   2517.8764    534.053      4.715      0.000    1471.034    3564.719
C(Total_Stops)[T.4 stops]   7091.8764   3568.608      1.987      0.047      96.741    1.41e+04
C(Total_Stops)[T.non-stop] -5569.2232     76.882    -72.438      0.000   -5719.927   -5418.520
==============================================================================
Omnibus:                     8326.591   Durbin-Watson:                   1.996
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           543670.670
Skew:                           3.217   Prob(JB):                         0.00
Kurtosis:                      37.353   Cond. No.                         111.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [14]:
# Additional_Info
anova = ols('Price ~ C(Additional_Info)', data=df).fit()
anova.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.102
Model:                            OLS   Adj. R-squared:                  0.101
Method:                 Least Squares   F-statistic:                     134.7
Date:                Mon, 11 Mar 2019   Prob (F-statistic):          1.69e-241
Time:                        16:42:09   Log-Likelihood:            -1.0471e+05
No. Observations:               10683   AIC:                         2.094e+05
Df Residuals:                   10673   BIC:                         2.095e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
======================================================================================================================
                                                         coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------
Intercept                                           2.211e+04   1002.948     22.045      0.000    2.01e+04    2.41e+04
C(Additional_Info)[T.1 Short layover]               4633.3684   4485.321      1.033      0.302   -4158.696    1.34e+04
C(Additional_Info)[T.2 Long layover]                4370.3684   4485.321      0.974      0.330   -4421.696    1.32e+04
C(Additional_Info)[T.Business class]                 3.47e+04   2404.985     14.429      0.000       3e+04    3.94e+04
C(Additional_Info)[T.Change airports]              -5308.9173   1932.930     -2.747      0.006   -9097.820   -1520.014
C(Additional_Info)[T.In-flight meal not included]  -1.264e+04   1007.744    -12.541      0.000   -1.46e+04   -1.07e+04
C(Additional_Info)[T.No Info]                      -1.406e+04   2715.997     -5.179      0.000   -1.94e+04   -8741.106
C(Additional_Info)[T.No check-in baggage included] -1.846e+04   1032.294    -17.887      0.000   -2.05e+04   -1.64e+04
C(Additional_Info)[T.No info]                      -1.297e+04   1004.089    -12.915      0.000   -1.49e+04    -1.1e+04
C(Additional_Info)[T.Red-eye flight]               -1.124e+04   4485.321     -2.505      0.012      -2e+04   -2444.568
==============================================================================
Omnibus:                     3368.217   Durbin-Watson:                   2.005
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            24973.184
Skew:                           1.313   Prob(JB):                         0.00
Kurtosis:                      10.015   Cond. No.                         149.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Can't justify dropping any of these attributes just yet

## Execution

In [15]:
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,2019-03-24 00:00:00,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,2019-05-01 00:00:00,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,2019-06-09 00:00:00,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,2019-05-12 00:00:00,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,2019-03-01 00:00:00,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [16]:
def to_min(duration):
    """converts duration given in form '__h __m' to integer of min
    returns minutes if form is acceptable, else returns -1
    
    str -> int"""
    tot = 0
    
    try:
        hrs, mins = duration.split('h')
        tot += 60*int(hrs)
        if 'm' in mins:
            tot += int(mins[:-1])
        return tot
    except ValueError:
        return -1
    
    

In [18]:
df = df.drop(columns=['Arrival_Time', 'Route'])
df['Dep_Time'] = df['Dep_Time'].apply(lambda x: int(x[:2])//2 * 2)
# Convert 'Dep_Time' back to str to be handles as categorical feature
df = df.astype(dtype={'Dep_Time': 'str'})

df['Duration'] = df['Duration'].apply(lambda x: to_min(x))
# drop values that are corrupted
df = df.drop(index=df[df['Duration'] == -1].index)

# Deal with Date_of_Journey colum
# Possibly convert to continuos variable representing days in future
initial_date = pd.to_datetime('2019-03-01')
df['Date_of_Journey'] = (pd.to_datetime(df['Date_of_Journey']) - initial_date).dt.days
df = df.rename(columns={'Date_of_Journey': 'Days_from_Journey'})

# Split data into independent features X and dependent features y
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

X_encoded = pd.get_dummies(X, drop_first=True)

In [20]:
df = df.drop(index=X[X['Duration'] == -1].index)

In [21]:
# Split X into numeric and categorical features
X_num = X_encoded.iloc[:, :2].values
X_cat = X_encoded.iloc[:, 2:].values

# Setup numeric transformer
# This will usually entail handling missing variables then scaling
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

# Define the preprocessor
preprocessor = ColumnTransformer(transformers=[('num', num_transformer, list(range(0,X_num.shape[1])))], 
                                            remainder='passthrough')

# fit and transform X
X_train = preprocessor.fit_transform(X_encoded)

# Scale y
sc_y = StandardScaler()
y = y.values
y = sc_y.fit_transform(y.reshape(-1,1))
y = np.ravel(y)

/Users/jdkrahulik/anaconda3/envs/conda_root/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/jdkrahulik/anaconda3/envs/conda_root/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


# Apply ML

In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_squared_log_error

In [33]:
regressor = LinearRegression()
regressor.fit(X_train, y)

mean_squared_log_error(sc_y.inverse_transform(y), sc_y.inverse_transform(regressor.predict(X_train)))

0.06884717592078787

Pretty promising result, although since the model includes so many variables, it is pretty likely that the model is overfit to the training set.

In [26]:
from sklearn.feature_selection import RFE
import statsmodels.formula.api as sm

In [54]:
estimator = LinearRegression()
selector = RFE(estimator, step=1, )
selector = selector.fit(X_train, y)

opt = selector.get_support()

OLS = sm.OLS(y, X_train[:, opt]).fit()
OLS.summary()

mean_squared_log_error(sc_y.inverse_transform(y), sc_y.inverse_transform(selector.predict(X_train)))

0.0756897376816206

Pretty good result albeit yielding a slightly higher error than the previous model. This model has reduced the number of features in half, and so is a much more general model. 

In [46]:
from sklearn.feature_selection import RFECV

In [48]:
y_true = sc_y.inverse_transform(y)

In [80]:
estimator = LinearRegression()
selector = RFECV(estimator, cv=5, step=1, scoring='neg_mean_squared_error', min_features_to_select=15)
selector = selector.fit(X_train, y)

opt = selector.get_support()

# OLS = sm.OLS(sc_y.inverse_transform(y), X_train[:, opt]).fit()
OLS = sm.OLS(sc_y.inverse_transform(y), np.concatenate([X_train[:, opt], np.ones(shape=(len(X_train[:, opt]), 1))], axis=1)).fit()
OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.689
Model:                            OLS   Adj. R-squared:                  0.688
Method:                 Least Squares   F-statistic:                     673.5
Date:                Tue, 12 Mar 2019   Prob (F-statistic):               0.00
Time:                        09:15:21   Log-Likelihood:                -99036.
No. Observations:               10682   AIC:                         1.981e+05
Df Residuals:                   10646   BIC:                         1.984e+05
Df Model:                          35                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          -538.1573     28.565    -18.840      0.000    -594.150    -482.165
x2          1772.9926    136.777     12.963      0.000    1504.884    2041.101
x3           203.6054    129.333      1.574      0.115     -49.912     457.123
x4          6053.4847    132.478     45.694      0.000    5793.804    6313.166
x5          4.085e+04   1337.081     30.548      0.000    3.82e+04    4.35e+04
x6          3671.6463    141.861     25.882      0.000    3393.573    3949.720
x7          3421.6018    726.056      4.713      0.000    1998.396    4844.807
x8          -292.5151    166.105     -1.761      0.078    -618.111      33.081
x9         -2920.6046   2584.633     -1.130      0.259   -7986.968    2145.758
x10         2116.9849    165.179     12.816      0.000    1793.203    2440.767
x11         3727.4075   1493.180      2.496      0.013     800.495    6654.320
x12          313.2831    153.272      2.044      0.041      12.841     613.726
x13         -678.5684     61.323    -11.065      0.000    -798.773    -558.364
x14         -496.5330    106.606     -4.658      0.000    -705.501    -287.565
x15         -678.5684     61.323    -11.065      0.000    -798.773    -558.364
x16         1319.6547    105.173     12.547      0.000    1113.496    1525.814
x17         -227.0361     87.046     -2.608      0.009    -397.662     -56.410
x18         -252.3244    114.450     -2.205      0.027    -476.668     -27.981
x19         -543.0131     90.072     -6.029      0.000    -719.571    -366.456
x20         -303.2075    181.927     -1.667      0.096    -659.818      53.403
x21         -437.1805    100.518     -4.349      0.000    -634.214    -240.147
x22         -590.5181     75.954     -7.775      0.000    -739.402    -441.634
x23         1983.8140     84.085     23.593      0.000    1818.992    2148.636
x24         3311.2951    391.718      8.453      0.000    2543.455    4079.135
x25         2533.3116   2784.337      0.910      0.363   -2924.508    7991.132
x26        -2059.3192   2577.399     -0.799      0.424   -7111.502    2992.864
x27        -3427.8388     89.824    -38.162      0.000   -3603.911   -3251.767
x28         5008.8078   2644.764      1.894      0.058    -175.423    1.02e+04
x29         5336.3259   2645.683      2.017      0.044     150.293    1.05e+04
x30         7014.3295   1735.026      4.043      0.000    3613.355    1.04e+04
x31        -4160.5093   1208.995     -3.441      0.001   -6530.365   -1790.653
x32        -1.158e+04    600.466    -19.279      0.000   -1.28e+04   -1.04e+04
x33        -9990.5975   1602.633     -6.234      0.000   -1.31e+04   -6849.138
x34        -8098.4533    625.755    -12.942      0.000   -9325.049   -6871.858
x35        -7913.5881    598.130    -13.231      0.000   -9086.035   -6741.141
x36        -3676.5467   2647.143     -1.389      0.165   -8865.442    1512.349
co

In [79]:
np.concatenate([X_train[:, opt], np.ones(shape=(len(X_train[:, opt]), 1))], axis=1)

array([[-1.15175689,  0.        ,  1.        , ...,  1.        ,
         0.        ,  1.        ],
       [-0.10562219,  1.        ,  0.        , ...,  1.        ,
         0.        ,  1.        ],
       [ 0.96804236,  0.        ,  0.        , ...,  1.        ,
         0.        ,  1.        ],
       ...,
       [-0.21574163,  0.        ,  0.        , ...,  1.        ,
         0.        ,  1.        ],
       [-1.78494367,  0.        ,  0.        , ...,  1.        ,
         0.        ,  1.        ],
       [ 0.11461669,  1.        ,  0.        , ...,  1.        ,
         0.        ,  1.        ]])

In [72]:
mean_squared_log_error(sc_y.inverse_transform(y), sc_y.inverse_transform(selector.predict(X_train)))

0.06866909107237816

In [70]:
X_encoded.columns[opt]

Index(['Days_from_Journey', 'Airline_Air India', 'Airline_IndiGo',
       'Airline_Jet Airways', 'Airline_Jet Airways Business',
       'Airline_Multiple carriers',
       'Airline_Multiple carriers Premium economy', 'Airline_SpiceJet',
       'Airline_Trujet', 'Airline_Vistara', 'Airline_Vistara Premium economy',
       'Source_Chennai', 'Source_Mumbai', 'Destination_Delhi',
       'Destination_Hyderabad', 'Destination_New Delhi', 'Dep_Time_10',
       'Dep_Time_12', 'Dep_Time_18', 'Dep_Time_2', 'Dep_Time_4', 'Dep_Time_6',
       'Total_Stops_2 stops', 'Total_Stops_3 stops', 'Total_Stops_4 stops',
       'Total_Stops_nan', 'Total_Stops_non-stop',
       'Additional_Info_1 Short layover', 'Additional_Info_2 Long layover',
       'Additional_Info_Business class', 'Additional_Info_Change airports',
       'Additional_Info_In-flight meal not included',
       'Additional_Info_No Info',
       'Additional_Info_No check-in baggage included',
       'Additional_Info_No info', 'Additional_Inf

In [84]:
from sklearn.ensemble import RandomForestRegressor

In [170]:
RFR_reg = RandomForestRegressor(n_estimators=200, criterion='mse', max_features=None)
RFR_reg.fit(X_train, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=None, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [171]:
mean_squared_log_error(sc_y.inverse_transform(y), sc_y.inverse_transform(RFR_reg.predict(X_train)))

0.003416849852402338

In [138]:
len(RFR_reg.feature_importances_)

47

Result above is too good to be true. Surely overfitted.

# ====================================================

In [112]:
# Load Test Data 

# set path, read file to constuct pandas dataframe
path_test = r'20190312_Test_set.xlsx'
df_test = pd.read_excel(path_test, header=0, dtype='str')

df_test = df_test.drop(columns=['Arrival_Time', 'Route'])
df_test['Dep_Time'] = df_test['Dep_Time'].apply(lambda x: int(x[:2])//2 * 2)
# Convert 'Dep_Time' back to str to be handles as categorical feature
df_test = df_test.astype(dtype={'Dep_Time': 'str'})

df_test['Duration'] = df_test['Duration'].apply(lambda x: to_min(x))
# drop values that are corrupted
df_test = df_test.drop(index=df[df['Duration'] == -1].index)

# Deal with Date_of_Journey colum
# Possibly convert to continuos variable representing days in future
initial_date = pd.to_datetime('2019-03-01')
df_test['Date_of_Journey'] = (pd.to_datetime(df_test['Date_of_Journey']) - initial_date).dt.days
df_test = df_test.rename(columns={'Date_of_Journey': 'Days_from_Journey'})

# Split data into independent features X and dependent features y
X_test = df_test.iloc[:, :]

X_test_encoded = pd.get_dummies(X_test, drop_first=True)

# Split X into numeric and categorical features
X_test_num = X_test_encoded.iloc[:, :2].values
X_test_cat = X_test_encoded.iloc[:, 2:].values

# Setup numeric transformer
# This will usually entail handling missing variables then scaling
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

# Define the preprocessor
preprocessor = ColumnTransformer(transformers=[('num', num_transformer, list(range(0,X_test_num.shape[1])))], 
                                            remainder='passthrough')

# fit and transform X
X_test = preprocessor.fit_transform(X_test_encoded)

In [158]:
# Test Data does not contain the same columns as Training data.
# Refit model considering only features that are contained in the test set

X_mask = X_encoded[mask]

# Split X into numeric and categorical features
X_num_mask = X_mask.iloc[:, :2].values
X_cat_mask = X_encoded.iloc[:, 2:].values

# Setup numeric transformer
# This will usually entail handling missing variables then scaling
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

# Define the preprocessor
preprocessor = ColumnTransformer(transformers=[('num', num_transformer, list(range(0,X_num_mask.shape[1])))], 
                                            remainder='passthrough')

# fit and transform X
X_train_mask = preprocessor.fit_transform(X_mask)

In [172]:
RFR_reg.fit(X_train_mask, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=None, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [175]:
# Attempt 2
price_2 = pd.Series(sc_y.inverse_transform(RFR_reg.predict(X_test)), name='Price_2')

In [163]:
sc_y.inverse_transform(RFR_reg.predict(X_test))

array([14799.74,  4286.88, 12898.  , ..., 16814.8 , 11712.56,  7416.76])

In [164]:
pd.Series(sc_y.inverse_transform(RFR_reg.predict(X_test))).to_csv('Test_Predicitons.csv')

In [166]:
predictions = pd.DataFrame(data=sc_y.inverse_transform(RFR_reg.predict(X_test)), columns=['Price'])

In [182]:
updated_pred = pd.concat([predictions, price_2], axis=1)
updated_pred['Difference'] = updated_pred['Price'] - updated_pred['Price_2']

In [183]:
updated_pred.sort_values('Difference')

,Price,Price_2,Difference
1819,28531.107576,32417.823500,-3886.715924
1632,15155.826476,16931.924167,-1776.097690
2149,9755.140000,11413.345000,-1658.205000
407,9280.735333,10880.557548,-1599.822214
907,6406.540000,7873.065000,-1466.525000
1180,18619.100000,19953.350000,-1334.250000
1093,16033.460000,17287.970000,-1254.510000
713,12577.857143,13707.502500,-1129.645357
1762,8739.025333,9767.266298,-1028.240964
151,14876.450000,15863.742333,-987.292333


In [185]:
updated_pred['Price_2'].to_csv('20190312_Submission_2.csv')

In [168]:
predictions['Price'].describe()

count     2671.000000
mean      9103.031864
std       4371.562595
min       1812.880000
25%       5465.766667
50%       8708.440000
75%      12378.958333
max      58203.660000
Name: Price, dtype: float64

In [169]:
df['Price'].describe()

count    10682.000000
mean      9086.292735
std       4610.885695
min       1759.000000
25%       5277.000000
50%       8372.000000
75%      12373.000000
max      79512.000000
Name: Price, dtype: float64

# ====================================================

In [ ]:
# Scratch work

def to_min(duration):
    """converts duration given in form '__h __m' to integer of min
    
    str -> int"""
    tot = 0
    
    try:
        hrs, mins = duration.split('h')
        tot += 60*int(hrs)
        if 'm' in mins:
            tot += int(mins[:-1])
        return tot
    except ValueError:
        return int(duration[:-1])
    
    

In [ ]:
initial_date = pd.to_datetime('2019-03-01')
X['Date_of_Journey'] = (pd.to_datetime(X['Date_of_Journey']) - initial_date).dt.days

In [150]:
mask = []

for col in X_encoded.columns:
    if col in X_test_encoded.columns:
        mask.append(col)
print(np.array(mask))

['Days_from_Journey' 'Duration' 'Airline_Air India' 'Airline_GoAir'
 'Airline_IndiGo' 'Airline_Jet Airways' 'Airline_Jet Airways Business'
 'Airline_Multiple carriers' 'Airline_Multiple carriers Premium economy'
 'Airline_SpiceJet' 'Airline_Vistara' 'Airline_Vistara Premium economy'
 'Source_Chennai' 'Source_Delhi' 'Source_Kolkata' 'Source_Mumbai'
 'Destination_Cochin' 'Destination_Delhi' 'Destination_Hyderabad'
 'Destination_Kolkata' 'Destination_New Delhi' 'Dep_Time_10' 'Dep_Time_12'
 'Dep_Time_14' 'Dep_Time_16' 'Dep_Time_18' 'Dep_Time_2' 'Dep_Time_20'
 'Dep_Time_22' 'Dep_Time_4' 'Dep_Time_6' 'Dep_Time_8'
 'Total_Stops_2 stops' 'Total_Stops_3 stops' 'Total_Stops_4 stops'
 'Total_Stops_non-stop' 'Additional_Info_Business class'
 'Additional_Info_Change airports'
 'Additional_Info_In-flight meal not included'
 'Additional_Info_No check-in baggage included' 'Additional_Info_No info']


In [152]:
mask == X_test_encoded.columns.values

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [154]:
X_encoded[mask]

,Days_from_Journey,Duration,Airline_Air India,Airline_GoAir,Airline_IndiGo,Airline_Jet Airways,Airline_Jet Airways Business,Airline_Multiple carriers,Airline_Multiple carriers Premium economy,Airline_SpiceJet,...,Dep_Time_8,Total_Stops_2 stops,Total_Stops_3 stops,Total_Stops_4 stops,Total_Stops_non-stop,Additional_Info_Business class,Additional_Info_Change airports,Additional_Info_In-flight meal not included,Additional_Info_No check-in baggage included,Additional_Info_No info
0,23,170,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
1,61,445,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
2,100,1140,0,0,0,1,0,0,0,0,...,1,1,0,0,0,0,0,0,0,1
3,72,325,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,285,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,115,145,0,0,0,0,0,0,0,1,...,1,0,0,0,1,0,0,0,0,1
6,11,930,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
7,0,1265,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
8,11,1530,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
9,87,470,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
